# 뉴스 요약하고 워드파일로 저장하기

In [1]:
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key = api_key
)

In [2]:
summary_prompt = """당신은 언어 이해 및 요약에 훈련된 고도로 숙련된 AI입니다.
다음 텍스트를 읽고 간결한 추상적 단락으로 요약했으면 합니다.
전체 텍스트를 읽을 필요 없이 토론의 요점을 이해하는 데 도움이 될 수 있는 일관되고
읽을 수 있는 요약을 제공하여 가장 중요한 요점을 유지하는 것을 목표로 합니다.
불필요한 세부 사항이나 접선 사항은 피하십시오."""

key_points_prompt = """당신은 정보를 핵심 포인트로 전달하는 데 특화된 능숙한 AI입니다.
다음 텍스트를 기반으로 논의되거나 언급된 주요 포인트를 확인하고 나열합니다.
이는 논의의 본질에 가장 중요한 아이디어, 결과 또는 주제가 되어야 합니다.
당신의 목표는 누군가가 읽을 수 있는 목록을 제공하여 이야기된 내용을 빠르게 이해하는 것입니다."""

action_items_prompt = """당신은 대화를 분석하고 행동 항목을 추출하는 데 있어 AI 전문가입니다.
본문을 검토하고 합의되거나 수행이 필요하다고 언급된 모든 작업, 과제 또는 행동을 식별하십시오.
이것들은 특정 개인에게 할당된 작업일 수도 있고 그룹이 취하기로 결정한 일반적인 행동일 수도 있습니다.
이러한 행동 항목을 명확하고 간결하게 나열하십시오."""

sentiment_prompt = """당신은 언어와 감정 분석에 전문성을 갖춘 AI로서 당신의 과제는 다음 텍스트의 감정을 분석하는 것입니다.
토론의 전체적인 톤, 사용된 언어가 전달하는 감정, 단어와 구가 사용되는 맥락을 고려하십시오.
감정이 일반적으로 긍정적인지 부정적인지 중립적인지를 표시하고 가능한 한 당신의 분석에 대해 간략한 설명을 제공하십시오."""


In [3]:
audio_file_path = "news.mp3"

with open(audio_file_path, 'rb') as audio_file:
     news_text = client.audio.transcriptions.create(
          model="whisper-1",
          file=audio_file,
          response_format="text"
     )

In [4]:
def text_extraction(text, prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content


In [5]:
abstract_summary = text_extraction(news_text, summary_prompt)
key_points = text_extraction(news_text, key_points_prompt)
action_items = text_extraction(news_text, action_items_prompt)
sentiment = text_extraction(news_text, sentiment_prompt)

In [6]:
news_data = {
    'Abstract Summary': abstract_summary,
    'Key Points': key_points,
    'Action Items': action_items,
    'Sentiment': sentiment
}

print(news_data)

{'Abstract Summary': '짧은 장마와 긴 폭염으로 인해 말벌의 활동이 증가하고 있으며, 이로 인해 119 소방대원의 출동이 빈번해지고 있습니다. 최근에는 하루에 최대 15건의 벌집 제거 요청이 접수되고 있으며, 지난달에는 벌집 제거 출동이 지난해보다 57% 증가한 4만 4천 건에 달했습니다. 소방대원들은 보호복을 착용하고 안전하게 벌집을 제거하며, 시민들에게는 주변에 벌집이 있는지 확인하고, 벌이 공격할 경우 20미터 이상 거리에서 대피할 것을 권장하고 있습니다.', 'Key Points': '1. **말벌의 극성**: 짧은 장마와 긴 폭염으로 말벌의 활동이 증가하고 있음.\n2. **소방대원의 출동 증가**: 119 소방대원들이 벌집 제거 작업에 바빠지고 있으며, 수요일에는 최대 15건까지 출동.\n3. **벌집 위치**: 신고된 벌집은 사무실 창문 바로 위와 같은 인근 건물에 위치.\n4. **벌 공격 습성**: 말벌은 어두운 색을 공격하는 경향이 있어, 소방대원들은 하얀색 보호복을 착용.\n5. **벌집 제거 방법**: 소방대원들은 스프레이를 사용해 벌집을 제거하고, 사람들을 대피시킴.\n6. **벌 쏘임 사고 증가**: 8월과 9월에 벌 쏘임 사고가 가장 많이 발생하고 있음.\n7. **벌집 제거 출동 통계**: 지난달 소방에 벌집 제거 출동 건수는 4만 4천 건으로, 지난해보다 57% 증가.\n8. **소방당국의 조언**: 주변에 벌집이 있는지 확인하고, 벌이 공격할 경우 20미터 거리까지 빠르게 벗어나라고 권장.', 'Action Items': '1. 119 소방대원들은 말벌 제거 작업을 수행한다.\n2. 소방대원들은 보호복을 착용하고 벌집 제거를 위해 스프레이를 사용한다.\n3. 소방대원들은 사람들을 대피시킨 후 벌집을 떼어낸다.\n4. 소방대원들은 벌이 공격해올 때 20미터 거리까지 빨리 벗어나라고 조언한다.\n5. 소방당국은 주변에 벌집이 있는지 잘 살펴보고 신고할 것을 권장한다.\n6. 소방대원들은 벌집 제거 출동 건수가 증가하

In [7]:
from docx import Document

doc = Document()

for key, value in news_data.items():
     # _로 나눈 단어들을 대문자로 만들고 공백으로 연결
     # heading = ' '.join(word.capitalize() for word in key.split('_'))
     doc.add_heading(key, level=1)
     doc.add_paragraph(value)
     # 섹션 사이에 줄바꿈 추가
     doc.add_paragraph()

doc.save('news_info.docx')
